# Indexing

## <mark>1. Documents Loading</mark>

LangChain 提供了各种 **Document Loaders（文档加载器）**，它们的作用是提供一个标准接口，将来自不同来源（文件系统、网页、数据库、SaaS 服务等）的数据读取并转换为 LangChain 的标准 `Document` 格式。

一个 `Document` 对象通常包含：

  * `page_content`: 数据的文本内容。
  * `metadata`: 可选的元数据，如源文件名、URL、页码等。

以下是一些常见文件类型的加载方式：

### 📁 常见本地文件加载器

| 文件类型 | LangChain 加载器 | 描述 |
| :--- | :--- | :--- |
| **纯文本** (`.txt`, `.md`) | `TextLoader` | 用于加载简单的纯文本文件或 Markdown 文件。 |
| **CSV** (`.csv`) | `CSVLoader` | 用于从 CSV 文件加载数据，可以配置列提取，通常每一行转换为一个 `Document`。 |
| **PDF** (`.pdf`) | `PyPDFLoader` | 使用 `pypdf` 库来加载和解析 PDF 文件，简单易用。 |
| **JSON** (`.json`) | `JSONLoader` | 用于加载 JSON 文件，通常需要指定 `jq` 路径来提取内容。 |
| **DOCX/Word** | `Docx2txtLoader` / `UnstructuredFileLoader` | 用于加载 Word 文档。`UnstructuredFileLoader` 支持多种非结构化文件。 |
| **目录**（多种文件）| `DirectoryLoader` | 用于从一个文件夹中递归加载所有文件，可以通过 `glob` 参数指定文件类型，并可以指定内部使用的子加载器。 |

> **提示:** 对于复杂的、格式不固定的非结构化文件（如表格、图片混排的 PDF、Word 等），通常推荐使用 **`UnstructuredFileLoader`**，因为它整合了更强大的解析能力。

### 🌐 Web 和数据源加载器

| 来源 | LangChain 加载器 | 描述 |
| :--- | :--- | :--- |
| **单个网页** | `WebBaseLoader` | 使用 `urllib` 和 `BeautifulSoup` 来加载和解析单个 HTML 网页。 |
| **递归网页** | `RecursiveUrlLoader` | 用于递归加载网页链接，适用于爬取网站。 |
| **YouTube** | `YoutubeLoader` | 用于加载 YouTube 视频的字幕/脚本。 |
| **云存储** | `S3FileLoader`, `S3DirectoryLoader` 等 | 用于从 AWS S3、Azure Blob Storage、Google Cloud Storage 等云服务加载文件。 |
| **SaaS/工具** | `NotionDirectoryLoader`, `SlackDirectoryLoader` 等 | 针对 Notion、Slack、GitHub 等应用集成的加载器。 |

In [ ]:
# LangChain Document Loaders 示例

# 1. 纯文本(.txt, .md) - TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader("document.txt", encoding="utf-8")
documents = loader.load()


# 2. CSV 文件 - CSVLoader
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path='data.csv',
    csv_args={
        'delimiter': ','
    }
)
documents = loader.load()
# print(f"加载的文档数量 (行数): {len(documents)}")
# print(documents[0].page_content) # 包含指定的列内容
# print(documents[0].metadata) # 自动包含 {'source': 'data.csv', 'row': 0}


# 3. 网页内容 - WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.example.com/some_article")
documents = loader.load()


# 4. 目录（多文件类型） - DirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# 实例化 DirectoryLoader，指定目录路径和要加载的文件类型（通过 glob）
# 通过 loader_cls 指定对匹配文件使用哪种 DocumentLoader
loader = DirectoryLoader(
    path='./documents',
    glob='**/*.txt',         # 只加载 .txt 文件
    loader_cls=TextLoader,   # 使用 TextLoader 来处理匹配到的 .txt 文件
    show_progress=True       # 显示加载进度
)
documents = loader.load()


# 5. DOCX/Word 文件 - UnstructuredFileLoader

# 对于 Word 文件，常用的加载器是 UnstructuredFileLoader，因为它能够处理各种非结构化文档，包括 Word、PPT、Email 等，并且解析效果通常比简单加载器好。
# 对于简单的 .docx，有时也可以使用更轻量级的 Docx2txtLoader。

from langchain_community.document_loaders import UnstructuredFileLoader

loader = UnstructuredFileLoader("report.docx")
documents = loader.load()


# 6. PDF 文件 - PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("manual.pdf")
# 加载文档。通常 PDF 的每一页会成为一个单独的 Document
documents = loader.load()

---

## <mark>2. Text Splitting</mark>

LangChain 提供了多种灵活的内置切分器，用于处理不同类型和格式的文本。

所有的文本切分器都继承自 TextSplitter 基类，它们的核心目标是将大块文本（或 Document）切割成更小的、有意义的块（Chunk），并保持块之间的上下文和元数据。

### 核心切分器

| 工具名称 | 适用场景 | 关键机制 | 输入类型 |
| :--- | :--- | :--- | :--- |
| **`RecursiveCharacterTextSplitter` (递归字符切分器)** | **最常用和推荐的**。适用于大多数通用文本。 | 尝试使用一连串字符（如 `\n\n`, `\n`, `     `, `.`）进行递归分割，直到满足块大小要求。优先使用更大的分隔符，最大限度地保留语义。 | `str` 或 `List[Document]` |
| **`CharacterTextSplitter` (字符切分器)** | 基础切分，按固定字符切割。 | 仅使用一个指定的分隔符（默认是 `\n\n`）进行切割。如果切割后的块超出大小，会再按长度硬性分割。 | `str` 或 `List[Document]` |

### 结构化和语义切分器

| 工具名称 | 适用场景 | 关键机制 | 输入类型 |
| :--- | :--- | :--- | :--- |
| **`MarkdownHeaderTextSplitter` (Markdown 头部切分器)** | **处理 Markdown 文件**。 | 根据用户定义的标题层级（如 `#`, `##`, `###`）进行切分，并将头部信息提取为子块的元数据。 | `str` |
| **`HTMLHeaderTextSplitter` (HTML 头部切分器)** | **处理 HTML 文件**。 | 类似 Markdown 切分器，根据 HTML 标签（如 `<h1>`, `<h2>`）进行切分，并将标签内容作为元数据。 | `str` |
| **`SemanticChunker` (语义切分器)** | **需要语义精确匹配**。 | 利用嵌入模型计算相邻句子的语义相似度，在相似度差异大的地方切分，确保每个块内语义连贯。 | `str` |

### 详细用法和推荐策略

#### 1\. RecursiveCharacterTextSplitter (递归字符切分器)

这是处理任何非结构化文本（如大段文章、代码、普通文档）的**首选**。

**❓ 何时使用？** 当我们想要最大限度地保留语义连贯性时。

**⚙️ 关键参数：**

  * **`chunk_size`** (`int`): 每个块的最大字符数（或 Token 数，取决于您的计算方式）。
  * **`chunk_overlap`** (`int`): 相邻块之间重叠的字符数，用于在检索时提供额外的上下文。
  * **`separators`** (`List[str]`): 用于递归切割的一系列分隔符，会按顺序尝试。

**💡 示例：**

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""] # 优先段落，其次换行，再次空格
)

# 传入纯字符串
splits_from_str = text_splitter.split_text("这是一个很长的文本，包含多段内容...") 

# 或者传入 Document 列表
# documents = [Document(...), Document(...)] 
# splits_from_doc = text_splitter.split_documents(documents) 
```

---

#### 2\. CharacterTextSplitter (字符切分器)

**❓ 何时使用？** 当我们只需要按固定字符（如句号、逗号、特定符号）进行简单切割，且不担心硬性分割时。

**💡 示例：**

```python
from langchain_text_splitters import CharacterTextSplitter

# 设定按句号切分，如果切分后的块太大，再按大小硬切
text_splitter = CharacterTextSplitter(
    separator=".", # 仅按句号切分
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
```

#### 3\. MarkdownHeaderTextSplitter (Markdown 头部切分器)

**❓ 何时使用？** 专门用于处理 Markdown 文档，尤其当我们需要将标题作为元数据嵌入到切块中，以提高检索准确性时。

**💡 示例：**

```python
from langchain_text_splitters import MarkdownHeaderTextSplitter

md_content = "# 第一章\n\n## 1.1 介绍\n\n这是一段内容。\n\n# 第二章\n\n文本内容..."

headers_to_split_on = [
    ("#", "Header 1"),  # 将 # 标记的内容提取为元数据 key: 'Header 1'
    ("##", "Header 2")  # 将 ## 标记的内容提取为元数据 key: 'Header 2'
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

# 只能传入纯字符串
md_splits = markdown_splitter.split_text(md_content)

# 结果：每个 Document 会有 'Header 1' 和 'Header 2' 对应的元数据
```

---

### 📝 关于输入类型：`str` vs. `Document`

| 方法 | 输入类型 | 描述 |
| :--- | :--- | :--- |
| **`split_text(text: str)`** | **纯字符串 (`str`)** | 切分器会处理纯文本，并在输出时**自动将每个块封装成 `Document`**。这适用于您自己用 `f.read()` 读取内容的情况。 |
| **`split_documents(documents: List[Document])`** | **`Document` 列表** | 切分器会遍历输入列表中的每一个 `Document`，对每个 `Document` 的 `page_content` 进行切分，并将原始 `Document` 的 `metadata` 继承到所有的子块中。这适用于您使用 LangChain Loaders 加载完文件后的情况。 |

下面使用2种方法对markdown文档进行切分：

In [1]:
# 1. 直接读取文件内容到纯字符串，使用MarkdownHeaderTextSplitter进行切分

file_path = r"E:\users\hwy\LLMs-Technology-Projects\LangChain_AI_Learning\data\员工手册.md"

with open(file_path, "r", encoding="utf-8") as f:
    md_content = f.read()

from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3")
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split)
md_header_splits = markdown_splitter.split_text(md_content)

md_header_splits

e:\users\hwy\LLMs-Technology-Projects\LangChain_AI_Learning\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'Header 1': '员工手册', 'Header 2': '第一章 企业文化', 'Header 3': '公司简介'}, page_content='上海斯俊慕智能科技有限公司，成立于2015年1月9日，坐落于上海浦东。互联魔方致力于打造企业消费全程管理数字化中台。中台系统分为费控系统、商旅系统、采购系统、结算及对账系统等模块。通过费控系统掌握和发起费用资源、商旅和采购系统参与管理供应商、结算和对账系统提供数据管理、收付款、供应链金融等服务，实现大中型企业消费支出的全过程、全生命周期管理。'),
 Document(metadata={'Header 1': '员工手册', 'Header 2': '第一章 企业文化', 'Header 3': '公司组织架构图'}, page_content='```\n上海斯俊慕智能科技有限公司\n\n总经办\n\nCEO                COO              CTO\n\n财务部     行政人事部      销售部               技术部\n\n南方大区  北方大区  研发中心  运维中心  项目中心\n```'),
 Document(metadata={'Header 1': '员工手册', 'Header 2': '第一章 企业文化', 'Header 3': '员工守则'}, page_content='#### 十准  \n- 遵守公司各项规章制度及合同条款，服从领导指挥及公司安排。\n- 遵守劳动纪律，坚守工作岗位，严格落实请、销假制度。\n- 接人待客礼貌热情，不卑不亢，热心为客户服务。\n- 勤奋工作，努力学习，不断提高自身素质，大胆提出合理化建议。\n- 部门之间加强协作，发扬主动配合的团队精神，以公司的整体利益为重。\n- 同事之间和睦相处，相互尊重、加强团结，严于律己，宽以待人，热情主动地帮助他人解决困难。\n- 要有主人翁意识，爱护公司财物；厉行节约，不浪费。\n- 要勇于负责，承担重任，对工作结果负责。\n- 严守公司秘密，实事求是，不谎报、不瞒报，不散播有损公司或他人名誉的谣言。\n- 认真贯彻执行公司的“五正”原则，传递正能量，拒绝消极和负面的语言。  \n#### 十不准  \n- 不准迟到、早退、无故旷工，

In [15]:
# 2. 使用加载器加载为Document，再使用 RecursiveCharacterTextSplitter 切分
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader(r"E:\users\hwy\LLMs-Technology-Projects\LangChain_AI_Learning\data\员工手册.md", encoding="utf-8")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
text_splits = text_splitter.split_documents(documents)

print(len(text_splits))

68


---

## <mark>3. Embedding</mark>

这一步主要涉及两个紧密相关的操作：

1.  **Embedding（嵌入）：** 将文本块（`Document` 的 `page_content`）转换为高维度的数值向量。
2.  **Vector Store 存储：** 将这些向量及其对应的元数据和原始文本块存储起来，以便高效地进行相似度搜索。

LangChain 提供了一个统一的接口来处理这两部分。

### 步骤一：选择和实例化 Embedding Model (嵌入模型)

| 模型类型 | LangChain 类 | 描述 |
| :--- | :--- | :--- |
| **OpenAI/大厂** | `OpenAIEmbeddings`, `CohereEmbeddings` | 访问云服务商提供的嵌入模型（如 OpenAI 的 `text-embedding-3-small`）。通常性能高，但需要 API Key。 |
| **开源本地** | `HuggingFaceEmbeddings` | 使用 Hugging Face 上托管的本地模型（如 `bge-small-en` 或国内的 `m3e-base`）。可以离线运行，节省成本，但可能需要更多本地计算资源。 |

---
### 步骤二：选择和实例化 Vector Store (向量数据库)

向量数据库是 RAG 系统的核心，它负责存储向量并执行高效的相似度搜索。LangChain 支持数百种向量存储方案。

以下是几种常见的向量存储方法，从最简单到最复杂的生产级方案：

#### 1\. In-Memory 内存存储 (最简单，仅用于测试)

| 工具名称 | 描述 | 适用场景 |
| :--- | :--- | :--- |
| **`FAISS`** | Facebook AI Similarity Search。虽然是高性能库，但在 LangChain 中常被用于**本地内存存储**。不需服务器，数据不持久化。 | **快速验证、本地测试、无持久化需求**。 |

**💡 示例：使用 FAISS**

```python
# pip install faiss-cpu
from langchain_community.vectorstores import FAISS

# 假设已经有了切分好的 Document 列表
# embeddings 也是上一步实例化的 HuggingFaceEmbeddings 对象

# 1. 使用 from_documents 方法创建索引并存储
# FAISS 会自动调用 embeddings 对象将 Document 转换为向量
vector_store = FAISS.from_documents(
    documents=text_splits, 
    embedding=embeddings
)

# 2. 存储索引到磁盘 (可选，用于持久化)
vector_store.save_local("faiss_index_path")

# 3. 后续使用时加载索引 (用于检索阶段)
loaded_vectorstore = FAISS.load_local("faiss_index_path", embeddings)
```

---
#### 2\. 轻量级持久化本地存储

| 工具名称 | 描述 | 适用场景 |
| :--- | :--- | :--- |
| **`Chroma`** | 另一个流行的轻量级 Python 原生向量数据库。它支持**持久化**，可以将数据存储在本地文件夹中。 | **中小规模应用、原型开发、需要数据持久化的本地项目**。 |

**💡 示例：使用 Chroma**

```python
# pip install chromadb
from langchain_community.vectorstores import Chroma

persist_directory = './chroma_db'

# 1. 创建索引并存储到本地文件夹
vector_store = Chroma.from_documents(
    documents=text_splits,
    embedding=embeddings,
    persist_directory=persist_directory # 指定数据存储路径
)

# 2. 调用 persist() 确保数据写入磁盘
# vector_store.persist() # 某些版本需要手动调用

# 3. 后续使用时加载 (用于检索阶段)
loaded_vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
```

---
#### 3\. 生产级云或托管服务

| 工具名称 | 描述 | 适用场景 |
| :--- | :--- | :--- |
| **`Milvus/Zilliz`** | 专业的开源向量数据库，提供高吞吐、高可用和分布式集群能力。 | **大规模数据、高并发、生产环境、企业级 RAG 系统**。 |
| **`Pinecone/Weaviate`** | 托管式云原生向量数据库，简化了部署和运维。 | **需要简化运维、追求极致性能和可扩展性的云端 RAG**。 |

**💡 示例：使用 Milvus (或其他专业向量库)**

这类库通常需要先启动服务（本地 Docker 或云服务），然后通过连接参数进行连接。

```python
# pip install pymilvus zilliz-langchain
from langchain_community.vectorstores import Milvus

# Milvus/Zilliz 连接参数
CONNECTION_ARGS = {
    "host": "127.0.0.1", # Milvus 或 Zilliz 集群地址
    "port": "19530",
    # ... 其他认证参数
}
COLLECTION_NAME = "employee_handbook_index"

# 1. 存储 Document
vector_store = Milvus.from_documents(
    documents=text_splits,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_args=CONNECTION_ARGS
)

# 注意：Milvus 是自带持久化的，无需手动 save/load 即可通过连接信息访问。
```

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embed = HuggingFaceEmbeddings(
    model_name=r"E:\users\hwy\LLMs-Technology-Projects\models\Dmeta-embedding-zh",
    model_kwargs={'device': 'cpu'}
)

In [ ]:
# 使用 Faiss
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(text_splits, embedding=embed)
vector_store.save_local("./faiss_blog/staff_handbook_db")

In [ ]:
# 加载本地的Faiss向量文件，allow_dangerous_deserialization 用于控制是否允许在加载向量存储时进行潜在的危险反序列化操作。
vector_store = FAISS.load_local(embeddings=embed, folder_path="./faiss_blog/staff_handbook_db", allow_dangerous_deserialization=True)

# 将 FAISS 向量存储为一个 retriever（检索器），并为该检索其设置一些搜索相关的参数，k=3表示检索时返回最相似的3个文档
retriever = vector_store.as_retriever(serch_kwargs={'k': 3})

# 执行相似度搜索
query = "请问我们公司有没有病假？"
results = retriever.invoke(query)

for doc in results:
    print(doc.page_content, end="\n")

In [20]:
# 直接在向量库上搜索并返回相似度
results = vector_store.similarity_search_with_score(query, k=3)

for doc, score in results:
    print(f"Score: {score:.4f}\n{doc.page_content}\n\n")

Score: 0.5477
9、参加公司组织的会议、培训、学习、考试或其他团队活动，如有事请假的，必须提前向组织者或带队者请假。在规定时间内未到或早退的，按照本制度第五条、第六条、第七条规定处理;未经批准擅自不参加的，视为旷工，按照本制度第七条规定处理。

10、员工按规定享受探亲假、婚假、产育假时，必须凭有关证明资料报部门总批准;未经批准者按旷工处理。员工病假期间只发给基本工资。


Score: 0.6161
- 假别分为：病假、事假、婚假、产假、年假、工伤假、丧假等七种。凡发生以上假者取消当月全勤奖。

- 病假：指员工生病必须进行治疗而请的假别;病假必须持县级以上医院证明，无有效证明按旷工处理;出据虚假证明加倍处罚;病假每月2日内扣除50%的基本日工资;超过2天按事假扣薪。

- 事假：指员工因事必须亲自办理而请的假别;但全年事假累计不得超过30天，超过天数按旷工处理;事假按实际天数扣罚日薪。

- 婚假：指员工达到法定结婚年龄并办理结婚证明而请的假别;

- 年假：指员工在公司工作满一年后可享受5天带薪休假，可逐年递增，特殊情况根据工作能力决定;年假必须提前申报当年使用。


Score: 0.7493
11、员工的考勤情况，由各部门负责人进行监督、检查，部门负责人对本部门的考勤要秉公办事，认真负责。如有弄虚作假、包痹袒护迟到、早退、旷工员工的，一经查实，按处罚员工的双倍予以处罚。凡是受到本制度第五条、第六条、第七条规定处理的员工，取消本年度先进个人的评比资格。 公司员工上班期间严格执行考勤制度，本制度适用于我公司一般员工至部门经理。

12、请假制度

- 假别分为：病假、事假、婚假、产假、年假、工伤假、丧假等七种。凡发生以上假者取消当月全勤奖。




然后即可将这个检索过程封装为一个完整的外部工具：

In [21]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query"""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [22]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")

prompt = "你是一名助人为乐的助手，当用户提问有关员工福利、休假政策、绩效管理、招聘、入职以及其他与HR相关的话题时，可以调用retrieve_context工具来进行文档检索回答。其他问题则可以调用你的原始知识库进行回答。"

tools = [retrieve_context]

model = ChatOpenAI(
    model="gpt-5-mini",
    temperature=0
)

agent = create_agent(model, tools, system_prompt=prompt)

response = agent.invoke({
    "messages": [
        {"role": "user", "content": "请问公司有病假吗？"}
    ]
})

In [23]:
response

{'messages': [HumanMessage(content='请问公司有病假吗？', additional_kwargs={}, response_metadata={}, id='3d0ed9b5-28b3-410a-9388-0fed0cd1a3b4'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 198, 'total_tokens': 255, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnb7lVjkUiptSF3SuMA7TI8D7NKw7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b2a09-797d-7d50-9024-76f010d320e3-0', tool_calls=[{'name': 'retrieve_context', 'args': {'query': '公司病假 政策 病假是否有？ sick leave policy 公司 病假安排 法定 病假流程 病假工资 需要证明'}, 'id': 'call_ngplfTRxfRooExvdr9MOimSk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 198, 'output_tokens': 57, 

In [24]:
response["messages"][-1].content

'有的。根据公司员工手册的规定：\n\n- 公司有病假。病假属于七类假期之一（病假、事假、婚假、产假、年假、工伤假、丧假）。  \n- 请病假需持县级以上医院的医疗证明；无有效证明的按旷工处理，提供虚假证明会被加重处罚。  \n- 病假工资处理：病假期间只发基本工资；每月病假在2日以内的，按实际病假天数扣除50%的基本日工资；超过2天的按事假扣薪（事假按实际天数扣罚日薪）。  \n- 发生请假还会影响当月全勤奖（发生上述任何假别会取消当月全勤奖）。\n\n如果你需要请病假，建议按公司流程提前向部门主管或人力提出申请，并尽快提交医院证明。如需我帮你写请假条或说明提交材料，我可以协助。'